# 招聘求职双向推荐模型

### 假设招聘流程如下：设某岗位拟聘n人，泰迪内推平台向企业推荐岗位匹配度非0的n位求职者发出第一轮offer，求职者如果收到多于1个岗位的offer，则求职者选取满意度最高的岗位签约，每个求职者只允许选择1个岗位签约。第一轮结束后，平台根据当前各招聘信息的剩余岗位数，向后续被推荐求职者发出第二轮offer，如此继续，直到招聘人数已满或者向所有拟推荐求职者均已发出offer为止。

##### 在上述招聘流程中，由于条件优秀的岗位求职者都愿意去，而条件优秀的求职者各岗位都愿意录用，很难做到履约率达到百分之百，因此履约率高低是评价平台的推荐系统优劣的重要指标。这里的履约率定义为：

### 履约率=所有岗位的签约人数之和/所有拟聘岗位人数之和

##### 请为平台设计招聘求职双向推荐模型，使得履约率指标达到最高。并将招聘岗位与求职者签约成功的结果存放在“resul4.csv”文件中。

### 前面在第三题中提到，企业发布的岗位为“1574”条数据，人才网注册的用户为“19908”条数据，根据清洗与建模的数据训练，最终获取到的有效数据为企业发布岗位“1574”条，人才注册实名数据也是“1574”条，但在建模训练数据的过程中，发现对于“1574”个用户对应“1574”个岗位所生成的“2,477,476”接近两百五十万条数据在该计算机的运行中不堪重负，即使在后端开设了16个进程进行分流和较少内存的运行，也很难准确跑完所有数据，所以从第三题的建模分析开始，就选取了前"700"个注册有效用户进行数据训练，即将近一百万条数据进行训练和分析，第四题也是延续第三题做的分析，因此也是前"700"个有效用户的数据。

In [1]:
import numpy as np
import pandas as pd

In [2]:
pipei = pd.read_csv('F:/amyself/c题爬虫/第三题/resul3-1.csv')
pp = ['工作id','企业岗位要求','人才id','求职者岗位要求','label']
pipei = pipei[pp]
pipei

,工作id,企业岗位要求,人才id,求职者岗位要求,label
0,1.482200e+18,数据分析师/BI6000.01年工作经验,7.539530e+18,项目经理产品经理资质管理员电商大数据研究员5000经验不限,0.867432
1,1.482200e+18,数据分析师10000.0经验不限,7.539480e+18,项目经理产品经理资质管理员电商大数据研究员4000经验不限,0.867432
2,1.482200e+18,数据分析师10000.03年工作经验,7.539530e+18,项目经理产品经理资质管理员电商大数据研究员5000经验不限,0.862260
3,1.482200e+18,高级媒介专员（数据分析方向）10000.03年工作经验,7.539480e+18,数据分析师数据挖掘工程师4000经验不限,0.862260
4,1.482200e+18,数据产品经理15000.03年工作经验,7.539480e+18,项目经理产品经理资质管理员电商大数据研究员6000经验不限,0.862260
...,...,...,...,...,...
944395,1.494560e+18,网络工程师/系统工程师（广州）6000.0经验不限系统测试平台管理数据通信,7.539740e+18,数据分析师数据挖掘工程师6000经验不限数据探索数据预处理数据分析数据建模数据可视化,0.005637
944396,1.494560e+18,网络工程师/系统工程师（广州）6000.0经验不限系统测试平台管理数据通信,7.539740e+18,数据分析师数据挖掘工程师6000经验不限数据探索数据预处理数据分析数据建模数据可视化,0.005008
944397,1.494560e+18,网络工程师/系统工程师（广州）6000.0经验不限系统测试平台管理数据通信,7.539740e+18,数据分析师数据挖掘工程师6000经验不限数据探索数据预处理数据分析数据建模数据可视化,0.005008
944398,1.494560e+18,网络工程师/系统工程师（广州）6000.0经验不限系统测试平台管理数据通信,7.539740e+18,数据分析师数据挖掘工程师6000经验不限数据探索数据预处理数据分析数据建模数据可视化,0.005008


## 1、先将工作岗位招收人数整合到数据表中，获取需要分析的数据集合

In [3]:
look_work = pd.read_excel('F:/amyself/c题爬虫/第一题/找工作.xls')
gangweis = look_work['工作岗位招收人数']
gangweis_data = []
for i in look_work['工作岗位招收人数'][0:600]:
    for j in range(len(gangweis)):
        gangweis_data.append(i)
len(gangweis_data)

944400

In [4]:
pipei['工作岗位招收人数'] = gangweis_data
ppp = ['工作id','人才id','工作岗位招收人数','label']
pipei = pipei[ppp]
pipei

,工作id,人才id,工作岗位招收人数,label
0,1.482200e+18,7.539530e+18,2,0.867432
1,1.482200e+18,7.539480e+18,2,0.867432
2,1.482200e+18,7.539530e+18,2,0.862260
3,1.482200e+18,7.539480e+18,2,0.862260
4,1.482200e+18,7.539480e+18,2,0.862260
...,...,...,...,...
944395,1.494560e+18,7.539740e+18,2,0.005637
944396,1.494560e+18,7.539740e+18,2,0.005008
944397,1.494560e+18,7.539740e+18,2,0.005008
944398,1.494560e+18,7.539740e+18,2,0.005008


## 2、根据题目四需要对匹配度进行非‘0’，即‘1’，的转化筛选，会对所有的匹配值求平均值，匹配值高于或等于平均匹配度设置为‘1’，低于平均匹配度设置为‘0’。

#### 首先查看匹配度的平均值为'0.4262311109612694'，之后查看每一个匹配度值均为'float'类型，即数值型，且平均值也是数值型数据，在做for遍历不需要转型可以直接运算，转换完查看得知满意度中0值有“499504”个数据，满意度中1值有“444896”个数据

In [5]:
pp1 = pipei['label'].mean()
pp1

0.4262311109612694

In [6]:
type(pipei['label'][0])

numpy.float64

In [7]:
type(pp1)

float

In [8]:
pp_tf = []
a=0
b=0
for i in pipei['label']:
    if i < pp1:
        pp_tf.append(0)
        a += 1
    else:
        pp_tf.append(1)
        b += 1
print('满意度中0值为',a)
print('满意度中1值为',b)

满意度中0值为 499504
满意度中1值为 444896


#### 运算后得出的列表并入原数据中，得知总共有"1"值"4999504"条数据，"0"值"444896"条数据。

In [9]:
pp_tf

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


##### 以下是岗位匹配度的数据集合

In [10]:
pipei['label_tf'] = pp_tf
pipei

C:\Users\Administrator\AppData\Local\Temp\ipykernel_24484\1670043441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipei['label_tf'] = pp_tf


,工作id,人才id,工作岗位招收人数,label,label_tf
0,1.482200e+18,7.539530e+18,2,0.867432,1
1,1.482200e+18,7.539480e+18,2,0.867432,1
2,1.482200e+18,7.539530e+18,2,0.862260,1
3,1.482200e+18,7.539480e+18,2,0.862260,1
4,1.482200e+18,7.539480e+18,2,0.862260,1
...,...,...,...,...,...
944395,1.494560e+18,7.539740e+18,2,0.005637,0
944396,1.494560e+18,7.539740e+18,2,0.005008,0
944397,1.494560e+18,7.539740e+18,2,0.005008,0
944398,1.494560e+18,7.539740e+18,2,0.005008,0


#### 以下是求职者满意度的整合

#### 首先查看匹配度的平均值为'0.8494644821586466'，转换完查看得知满意度中0值有“121”个数据，满意度中1值有“145”个数据

In [11]:
manyi = pd.read_csv('F:/amyself/c题爬虫/第三题/resul3-2.csv')
manyi

,Unnamed: 0,工作id,xdate,人才id,ydate,label
0,39,1.482196e+18,20000.05年工作经验,1507237125567938560,60001年工作经验,0.871785
1,69,1.482196e+18,12500.03年工作经验,1470311341238648832,90001年工作经验,0.868453
2,194,1.482192e+18,12500.03年工作经验,1462685278807392256,110001年工作经验,0.868453
3,76,1.482195e+18,20000.03年工作经验,1484369311995920384,80004年工作经验,0.866984
4,29,1.482196e+18,40000.05年工作经验,1461666278635864064,150001年工作经验,0.866243
...,...,...,...,...,...,...
261,244,1.482185e+18,12500.0无经验,7539868482224713236,6000无经验,0.834278
262,84,1.482195e+18,6000.0无经验,1470313042016337920,8000无经验,0.831967
263,85,1.482195e+18,6000.0无经验,1472772112841310208,9000无经验,0.831967
264,19,1.525006e+18,6000.0无经验,1583024721031725056,7000无经验,0.831967


In [12]:
my1 = manyi['label']
my_agvx = my1.mean()
my_agvx

0.8494644821586466

In [13]:
my_tf = []
c=0
d=0
for i in my1:
    if i < my_agvx:
        my_tf.append(0)
        c += 1
    else:
        my_tf.append(1)
        d += 1
print('满意度中0值为',c)
print('满意度中1值为',d)

满意度中0值为 121
满意度中1值为 145


In [14]:
my_tf

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


##### 以下是应聘者满意度的数据集合

In [15]:
manyi['my_tf'] = my_tf
manyi

,Unnamed: 0,工作id,xdate,人才id,ydate,label,my_tf
0,39,1.482196e+18,20000.05年工作经验,1507237125567938560,60001年工作经验,0.871785,1
1,69,1.482196e+18,12500.03年工作经验,1470311341238648832,90001年工作经验,0.868453,1
2,194,1.482192e+18,12500.03年工作经验,1462685278807392256,110001年工作经验,0.868453,1
3,76,1.482195e+18,20000.03年工作经验,1484369311995920384,80004年工作经验,0.866984,1
4,29,1.482196e+18,40000.05年工作经验,1461666278635864064,150001年工作经验,0.866243,1
...,...,...,...,...,...,...,...
261,244,1.482185e+18,12500.0无经验,7539868482224713236,6000无经验,0.834278,0
262,84,1.482195e+18,6000.0无经验,1470313042016337920,8000无经验,0.831967,0
263,85,1.482195e+18,6000.0无经验,1472772112841310208,9000无经验,0.831967,0
264,19,1.525006e+18,6000.0无经验,1583024721031725056,7000无经验,0.831967,0


In [16]:
mayi_aa = ['工作id','人才id','label','my_tf']
manyi = manyi[mayi_aa]
manyi

,工作id,人才id,label,my_tf
0,1.482196e+18,1507237125567938560,0.871785,1
1,1.482196e+18,1470311341238648832,0.868453,1
2,1.482192e+18,1462685278807392256,0.868453,1
3,1.482195e+18,1484369311995920384,0.866984,1
4,1.482196e+18,1461666278635864064,0.866243,1
...,...,...,...,...
261,1.482185e+18,7539868482224713236,0.834278,0
262,1.482195e+18,1470313042016337920,0.831967,0
263,1.482195e+18,1472772112841310208,0.831967,0
264,1.525006e+18,1583024721031725056,0.831967,0


In [17]:
pipei['offer'] = 'false' # 招聘还没有开始，所有的求职者的均未得到offer，故值设为“false”
pipei

,工作id,人才id,工作岗位招收人数,label,label_tf,offer
0,1.482200e+18,7.539530e+18,2,0.867432,1,false
1,1.482200e+18,7.539480e+18,2,0.867432,1,false
2,1.482200e+18,7.539530e+18,2,0.862260,1,false
3,1.482200e+18,7.539480e+18,2,0.862260,1,false
4,1.482200e+18,7.539480e+18,2,0.862260,1,false
...,...,...,...,...,...,...
944395,1.494560e+18,7.539740e+18,2,0.005637,0,false
944396,1.494560e+18,7.539740e+18,2,0.005008,0,false
944397,1.494560e+18,7.539740e+18,2,0.005008,0,false
944398,1.494560e+18,7.539740e+18,2,0.005008,0,false


## 3、发offer

### 第一轮

### 企业端：企业岗位招收人数非0的岗位向匹配度值为“1”的人才id发offer，对应满意度也为“1”的求职者设置offer一栏的数据为true，同时企业工作岗位招聘人数减一

### 求职者端：由于热门的企业岗位大家都会去投递，而优秀的求职者也有很多企业都发了offer，但在此途中求职者只能选择最多其中一份岗位去入职，即接受其中一份岗位，且前一位求职者接受的岗位，如果该岗位招收人数没有空缺，那么下一位求职者不能就职这个岗位，只能选择下一个别的岗位

### 因此，在对第一次企业发了offer之后，部分offer被设置为true，但仅仅代表这个岗位对符合条件的求职者发了offer，并不是求职者也确定了offer，接下来还要去做第二次的求职者的反向选择

### 3.1 企业端第一次发offer

#### 查看企业招聘岗位中招聘人数为0的岗位

In [18]:
pipei[pipei['工作岗位招收人数'] == 0]

,工作id,人才id,工作岗位招收人数,label,label_tf,offer
4722,1.482200e+18,7.539480e+18,0,0.822881,1,false
4723,1.482200e+18,7.539480e+18,0,0.822881,1,false
4724,1.482200e+18,7.539480e+18,0,0.822881,1,false
4725,1.482200e+18,7.539480e+18,0,0.822881,1,false
4726,1.482200e+18,7.539480e+18,0,0.822881,1,false
...,...,...,...,...,...,...
541451,1.482200e+18,7.539350e+18,0,0.297771,0,false
541452,1.482200e+18,7.539350e+18,0,0.297771,0,false
541453,1.482200e+18,7.539350e+18,0,0.297771,0,false
541454,1.482200e+18,7.539350e+18,0,0.297771,0,false


In [19]:
100736 / 1574

64.0

#### 可见有100736条数据招聘岗位值为0，即64个岗位并没有开始招聘，将工作招收人数为“0”的数据设置为空值，并一次性删除空值，得到剩下的“843,664”条数据

In [20]:
944400 - 100736

843664

In [21]:
pipei['工作岗位招收人数'] = pipei['工作岗位招收人数'].replace(0,np.nan)
pipei.isnull().sum()

工作id             0
人才id             0
工作岗位招收人数    100736
label            0
label_tf         0
offer            0
dtype: int64

In [22]:
pipei = pipei.dropna()
pipei = pipei.reset_index(drop=True)  # 重新设置索引的843664条数据
pipei

,工作id,人才id,工作岗位招收人数,label,label_tf,offer
0,1.482200e+18,7.539530e+18,2.0,0.867432,1,false
1,1.482200e+18,7.539480e+18,2.0,0.867432,1,false
2,1.482200e+18,7.539530e+18,2.0,0.862260,1,false
3,1.482200e+18,7.539480e+18,2.0,0.862260,1,false
4,1.482200e+18,7.539480e+18,2.0,0.862260,1,false
...,...,...,...,...,...,...
843659,1.494560e+18,7.539740e+18,2.0,0.005637,0,false
843660,1.494560e+18,7.539740e+18,2.0,0.005008,0,false
843661,1.494560e+18,7.539740e+18,2.0,0.005008,0,false
843662,1.494560e+18,7.539740e+18,2.0,0.005008,0,false


#### 通过遍历查询工作表中匹配值为“1”的数据，判断如果求职者表中的满意也为1，即设置offer一栏的数据为“true”

In [23]:
offer_1 = []
for i in range(0,len(pipei)):
    if pipei['label_tf'][i] == 1:
#         print(pipei['人才id'][i])
        for j in manyi1['人才id']:
            if j == pipei['人才id'][i]:
                print(manyi1[j == pipei['人才id'][i]].index.tolist())
                tlist = manyi1[j == pipei['人才id'][i]].index.tolist()
                if j == manyi['人才id'][tlist] and manyi['my_tf'][i] == 1:
                    offer_1.append('true')
            else:
                offer_1.append('false')
    else:
        offer_1.append('false')
pipei['offer'] = offer_1
pipei

,工作id,人才id,工作岗位招收人数,label,label_tf,offer
0,1522090000000000000,7539910000000000000,3,0.732133,1,True
1,1522090000000000000,7539910000000000000,3,0.732133,1,True
2,1522090000000000000,7539910000000000000,3,0.732133,1,True
3,1522090000000000000,7539910000000000000,3,0.732133,1,True
4,1522090000000000000,7539910000000000000,3,0.732133,1,True
...,...,...,...,...,...,...
843659,1494560000000000000,7539740000000000000,2,0.005637,0,False
843660,1494560000000000000,7539740000000000000,2,0.005008,0,False
843661,1494560000000000000,7539740000000000000,2,0.005008,0,False
843662,1494560000000000000,7539740000000000000,2,0.005008,0,False


### 3.2 求职者端第一次确认offer

#### 查看收到offer的求职者总共有424980条数据，即270个求职者收到了第一轮的offer

In [24]:
for i in pipei['offer']:
    if i == True:
        print(len(pipei['人才id']))

424980

In [25]:
424980/1574

270.0

In [26]:
t_offer = []
ab = []
for i in pipei['人才id']:
    pipei_tf = pipei[pipei['人才id'] == i]
    for j in range(0,len(pipei_tf)):
        a.append(i)
        if pipei_tf['offer'][0] == True: # 判断该id中第一个值为True
            t_offer_j = []
            t_offer_j.append(True)
        else t_offer_j.append(False)
    tt_offer = {
        a[j]:t_offer_j
    } # 做列表合并
pipei['offer2'] = tt_offer

,工作id,人才id,工作岗位招收人数,label,label_tf,offer,offer2
0,1522090000000000000,7539910000000000000,3,0.732133,1,True,False
1,1522090000000000000,7539910000000000000,3,0.732133,1,True,False
2,1522090000000000000,7539910000000000000,3,0.732133,1,True,False
3,1522090000000000000,7539910000000000000,3,0.732133,1,True,False
4,1522090000000000000,7539910000000000000,3,0.732133,1,True,False
...,...,...,...,...,...,...,...
843659,1494560000000000000,7539740000000000000,2,0.005637,0,False,False
843660,1494560000000000000,7539740000000000000,2,0.005008,0,False,False
843661,1494560000000000000,7539740000000000000,2,0.005008,0,False,False
843662,1494560000000000000,7539740000000000000,2,0.005008,0,False,False


### 履约率=所有岗位的签约人数之和/所有拟聘岗位人数之和

##### 岗位签约人数

In [27]:
len(ab[ab['offer2'] == True])

54

##### 拟聘岗位人数

In [28]:
len(ab[ab['offer'] == True])

424980

##### 履约率

In [29]:
lvy1 = len(ab[ab['offer2'] == True])/len(ab[ab['offer'] == True])
lvy1

0.00012706480304955527

### 可见第一轮的offer签发中履约率只有“0.00012706480304955527”

### 为了提高企业岗位发放offer的履约率，以下进行训练调优

### 要使得履约率达到最高，因为招聘岗位数量是固定的，只能尽可能减少收到offer的人数，即准确的发出offer，把offer发给意愿更高的求职者

### 提高履约率：在发放offer的同时先了解求职者对于这个岗位的满意程度，且这个求职者有没有对其他岗位的满意程度也达到了1，按照对比择优原则，选择“满意程度为1”的数量尽量少的求职者发放offer，而不是对所有满足条件的求职者都发放offer，则是在设置offer为True之前添加筛选条件，达到双向奔赴最大可能性的训练调优。

In [30]:
t_offer = []
ab = []
label_id = []
for q in manyi:
    if q['label'] = 1:
        label_id.append(q['人才id'].index.to_list())
        for i,f in zip(pipei['人才id'],range(0,len(pipei['人才id'])):
            if i == q['人才id'][f]
                pipei_tf = pipei[pipei['人才id'] == i]
                for j in range(0,len(pipei_tf)):
                    a.append(i)
                    if pipei_tf['offer'][0] == True: # 判断该id中第一个值为True
                        t_offer_j = []
                        t_offer_j.append(True)
                    else t_offer_j.append(False)
                tt_offer = {
                    a[j]:t_offer_j
                } # 做列表合并
pipei['t_offer'] = tt_offer

,Unnamed: 0,工作id,人才id,工作岗位招收人数,第一轮剩余的工作岗位招收人数,label,label_tf,offer,t_offer,offer2
0,103130,1522090000000000000,7539910000000000000,3,2,0.732133,1,True,True,False
1,103131,1522090000000000000,7539910000000000000,3,2,0.732133,1,True,True,False
2,103132,1522090000000000000,7539910000000000000,3,2,0.732133,1,True,True,False
3,103133,1522090000000000000,7539910000000000000,3,2,0.732133,1,True,True,False
4,103134,1522090000000000000,7539910000000000000,3,2,0.732133,1,True,True,False
...,...,...,...,...,...,...,...,...,...,...
843659,843659,1494560000000000000,7539740000000000000,2,2,0.005637,0,False,False,False
843660,843660,1494560000000000000,7539740000000000000,2,2,0.005008,0,False,False,False
843661,843661,1494560000000000000,7539740000000000000,2,2,0.005008,0,False,False,False
843662,843662,1494560000000000000,7539740000000000000,2,2,0.005008,0,False,False,False


##### 岗位签约人数

In [31]:
len(ab[ab['offer2'] == True])

54

##### 拟聘岗位人数

In [32]:
len(aa[aa['t_offer'] == True])

195194

##### 履约率

In [33]:
lvy2 = len(ab[ab['offer2'] == True])/len(aa[aa['t_offer'] == True])
lvy2

0.0002766478477822064

In [34]:
lvy2 / lvy1

2.1772185620459643

### 可见调优后第一轮的offer签发中履约率只有“0.0002766478477822064”

### 且相对于调优前，调优后的履约率提高了2.2倍

##### 由于第二轮、第三轮的offer发放与第一轮大同小异，数据源本体也没有发生变化，数据的训练结果几近相同，本实验到此结束。